<sub>Developed by SeongKu Kang, August 2025 — Do not distribute</sub>

# 📘 Task 1: Product category classification with no label (Fixed BERT embeddings)

In this notebook, we consider a **realistic but challenging scenario**: what if we have **no labeled data at all**?

In many real-world applications, collecting labeled product-category data is expensive and slow.  
Here, we explore how to bootstrap a classification system without any human-provided labels.  
Your task is to fill in the blanks and design solutions for this "zero-label" setting.

---

## Key Ideas (Guidelines)

1. **Constructing Silver Labels**  
   Since we have no ground-truth labels, we must create *weak supervision signals*.  
   Possible strategies include:
   - **Text similarity:** Compare product titles/descriptions with category names using TF-IDF or embeddings.  
   - **Embedding similarity:** Use pre-trained language model embeddings for both products and labels, then assign the nearest label.  
   - **Ensemble approaches:** Combine multiple weak signals (e.g., weighted voting between text-based and embedding-based similarity).

2. **Learning with Silver Labels**  
   Once silver labels are generated, train a classifier as if they were real labels.  
   To improve robustness, you may consider various techniques that we learned, including (but not limited to):
   - **Self-training:** Train an initial model with silver labels, then use it to assign pseudo-labels to unlabeled data with high confidence.  
   - **Label embedding models:** Instead of treating labels as arbitrary IDs, use semantic embeddings of label names to guide classification (e.g., inner-product classifier).
   - **Consistency regularization:** Encourage the model to produce stable predictions under input perturbations (e.g., dropout noise, data augmentation). This helps prevent overfitting to noisy silver labels and promotes smoother decision boundaries.
   - **Stabilizing model prediction using Ensemble:** To mitigate the noise from weak or unstable supervision, you can stabilize predictions through ensembling techniques (e.g., Temporal ensemble via EMA, independent model ensemble).

---

## Your Tasks

1. Generate silver labels.
2. Train a classifier using these silver labels and various learning strategies.
  
💡 *Hint:* Think of this as "bootstrapping" the learning process — even noisy initial signals can become useful when combined with iterative refinement and stabilization techniques.


⚠️ **Note**: Do **NOT** use the labeled training set provided in the previous notebook.  
In this notebook, you must assume that **no labeled data exists**. Only the following resources are allowed:
- Product metadata (titles, descriptions, etc.)
- Category names

In [1]:
import json
from tqdm import tqdm
from pathlib import Path
from utils import * 
import copy

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split, ConcatDataset

device = "cuda:2" if torch.cuda.is_available() else "cpu"

In [2]:
# Default paths
ROOT = Path("dataset") # Root dataset directory
CORPUS_PATH = ROOT / "corpus.jsonl" # Product corpus file (JSON Lines): Each line contains a product ID and its associated text description.
EMB_PATH = ROOT / "corpus_bert_mean.pt"

# Task 1: Product category classification
LABEL_MAP_PATH = ROOT / "category_classification" 
LABEL2ID_PATH = LABEL_MAP_PATH / "label2labelid.json" 
ID2LABEL_PATH = LABEL_MAP_PATH / "labelid2label.json" 
PID2LABEL_TEST_PATH = LABEL_MAP_PATH / "pid2labelids_test.json" 
LABEL_EMB_PATH = LABEL_MAP_PATH / "category_labels_bert_mean.pt"

In [3]:
pid2text = load_corpus(CORPUS_PATH) # load corpus

label2id = load_json(LABEL2ID_PATH)
id2label = load_json(ID2LABEL_PATH)
pid2label_test = load_json(PID2LABEL_TEST_PATH)

# loading pre-trained embeddings
corpus_data = torch.load(EMB_PATH)  # {"ids": [...], "embeddings": Tensor}
pid_list = corpus_data["ids"]
pid2idx = {pid: i for i, pid in enumerate(pid_list)}
embeddings = corpus_data["embeddings"]

label_data = torch.load(LABEL_EMB_PATH)
label_emb = label_data["embeddings"].to(device)

In [ ]:
# ==========================================================
# Your Task: Do your magic below 
# ==========================================================

In [ ]:
# corpus load
pid2text = {}
with open(CORPUS_PATH, "r", encoding="utf-8") as f:
    for line in f:
        item = json.loads(line)
        pid2text[item["_id"]] = item["title"] + " " + item["text"]

import re

def preprocess_label_text(label_path_str):
    """
    Cleans label string by splitting on '>', '&', and other special characters,
    and returns a space-separated token string.
    """
    # Replace '>' and '&' with space
    cleaned = re.sub(r"[>&]", " ", label_path_str)
    # Remove any non-alphanumeric characters except spaces
    cleaned = re.sub(r"[^a-zA-Z0-9 ]", "", cleaned)
    # Collapse multiple spaces into one
    cleaned = re.sub(r"\s+", " ", cleaned).strip()
    return cleaned

import torch.nn.functional as F
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from tqdm import tqdm

def build_tfidf_vectorizer(label_texts):
    vectorizer = TfidfVectorizer()
    label_tfidf = vectorizer.fit_transform(label_texts)
    return vectorizer, label_tfidf

def compute_lexical_similarity(doc_text, vectorizer, label_tfidf):
    doc_vec = vectorizer.transform([doc_text])
    sims = cosine_similarity(doc_vec, label_tfidf)[0]
    return sims

def generate_silver_labels(pid2text, pid2idx, id2label):
    """
    Returns:
        silver_top1_labels (dict): pid -> int (label_id)
    """
    # === Step 1: Preprocess label texts ===
    label_ids = sorted(list(id2label.keys()), key=lambda x: int(x))
    label_texts = [preprocess_label_text(id2label[label_id]) for label_id in label_ids]

    # === Step 2: Build TF-IDF on label texts ===
    vectorizer, label_tfidf = build_tfidf_vectorizer(label_texts)

    silver_labels = {}

    for pid, text in tqdm(pid2text.items(), desc="Generating Lexical Top-1 Labels"):
        if pid not in pid2idx:
            continue

        lexical_sims = compute_lexical_similarity(text, vectorizer, label_tfidf)
        top1_label = int(np.argmax(lexical_sims))
        silver_labels[pid] = top1_label

    return silver_labels

silver_labels = generate_silver_labels(pid2text, pid2idx, id2label)
pid2label_train = silver_labels

Generating Lexical Top-1 Labels: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39452/39452 [00:36<00:00, 1094.31it/s]


In [5]:
class ProductCategoryEmbeddingDataset(Dataset):
    def __init__(self, pid2label, pid2idx, embeddings):
        self.pids = list(pid2label.keys())
        self.labels = [pid2label[pid] for pid in self.pids]
        self.indices = [pid2idx[pid] for pid in self.pids]
        self.embeddings = embeddings

    def __len__(self):
        return len(self.pids)

    def __getitem__(self, idx):
        emb = self.embeddings[self.indices[idx]]
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return {"X": emb, "y": label}

In [6]:
from sklearn.metrics import accuracy_score, f1_score

def evaluate(model, dataloader, device="cpu"):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for batch in dataloader:
            X = batch["X"].to(device)
            y = batch["y"].to(device)
            logits = model(X)
            preds = torch.argmax(logits, dim=1)
            all_preds.extend(preds.cpu().tolist())
            all_labels.extend(y.cpu().tolist())

    acc = accuracy_score(all_labels, all_preds)
    f1_macro = f1_score(all_labels, all_preds, average="macro", zero_division=0)
    f1_micro = f1_score(all_labels, all_preds, average="micro", zero_division=0)

    return {"accuracy": acc, "f1_macro": f1_macro, "f1_micro": f1_micro}

In [7]:
import torch.nn as nn

class BaseClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.linear = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        return self.linear(x)

In [ ]:
test_dataset = ProductCategoryEmbeddingDataset(pid2label_test, pid2idx, embeddings)
test_loader = DataLoader(test_dataset, batch_size=64)

input_dim = embeddings.shape[1]
num_classes = len(label2id)

all_pids = set(pid_list)
test_pids = set(pid2label_test.keys())

In [9]:
results_dict = {'valid':{}, 'test': {}}

In [ ]:
train_dataset = ProductCategoryEmbeddingDataset(pid2label_train, pid2idx, embeddings)

val_ratio = 0.2
val_size = int(len(train_dataset) * val_ratio)
train_size = len(train_dataset) - val_size

train_split, val_split = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_split, batch_size=32, shuffle=True)
val_loader = DataLoader(val_split, batch_size=64)

In [11]:
used_pids = set(pid2label_train.keys())
unlabeled_pids = all_pids - used_pids
unlabeled_ratio = len(unlabeled_pids) / len(all_pids)

print(f"[Unlabeled] {len(unlabeled_pids)} / {len(all_pids)} = {unlabeled_ratio:.2%}")

[Unlabeled] 0 / 39452 = 0.00%


In [14]:
model = BaseClassifier(input_dim, num_classes).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4)

In [15]:
best_val_acc = -1
best_model_state = None
patience = 10
patience_counter = 0

val_acc_list = []
test_acc_list = []

EPOCHS = 15

for epoch in range(1, EPOCHS + 1):
    model.train()
    total_loss = 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch}"):
        X = batch["X"].to(device)              # shape: (B, D)
        y = batch["y"].to(device)              # shape: (B, C), float, soft multi-label

        logits = model(X)
        loss = F.cross_entropy(logits, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"[Epoch {epoch}] Train Loss: {avg_loss:.4f}")

    # === Validation ===
    val_result = evaluate(model, val_loader, device=device)
    val_acc = val_result["accuracy"]
    val_acc_list.append(val_acc)

    is_improved = val_acc > best_val_acc
    print_eval_result(val_result, stage="val", is_improved=is_improved)

    # === Test ===
    test_result = evaluate(model, test_loader, device=device)
    test_acc = test_result["accuracy"]
    test_acc_list.append(test_acc)
    print_eval_result(test_result, stage="test")

    # === Update best model ===
    if is_improved:
        best_val_acc = val_acc
        best_model_state = model.state_dict()
        patience_counter = 0
    else:
        patience_counter += 1

    # === Early stopping ===
    if patience_counter >= patience:
        print(f"[Early Stopping] No improvement for {patience} consecutive epochs.")
        break

Epoch 1: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 987/987 [00:03<00:00, 277.05it/s]


[Epoch 1] Train Loss: 5.1853
[VAL ] Acc: 0.1635 | F1-macro: 0.0038 *
[TEST] Acc: 0.0025 | F1-macro: 0.0011


Epoch 2: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 987/987 [00:03<00:00, 272.92it/s]


[Epoch 2] Train Loss: 4.4797
[VAL ] Acc: 0.1880 | F1-macro: 0.0117 *
[TEST] Acc: 0.0099 | F1-macro: 0.0049


Epoch 3: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 987/987 [00:03<00:00, 275.17it/s]


...

[Epoch 13] Train Loss: 2.5351
[VAL ] Acc: 0.3653 | F1-macro: 0.1737 *
[TEST] Acc: 0.1678 | F1-macro: 0.1342


Epoch 14: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 987/987 [00:03<00:00, 278.76it/s]


[Epoch 14] Train Loss: 2.4619
[VAL ] Acc: 0.3804 | F1-macro: 0.1942 *
[TEST] Acc: 0.1775 | F1-macro: 0.1433


Epoch 15: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 987/987 [00:03<00:00, 276.06it/s]


[Epoch 15] Train Loss: 2.3952
[VAL ] Acc: 0.3888 | F1-macro: 0.2028 *
[TEST] Acc: 0.1847 | F1-macro: 0.1519
